# 2. Building a Simple Convolutional Neural Network and using it to classify MNIST dataset

In [1]:
# Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [4]:
# Create a simple CNN
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x

# model = CNN()
# x = torch.randn(64, 1, 28, 28)
# print(model(x).shape)

In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [6]:
# Hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 64
IN_CHANNEL = 1
NUM_CLASSES = 10
EPOCHS = 10

In [7]:
# Load Data
train_data = datasets.MNIST(root='data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)
test_data = datasets.MNIST(root='data',
                           train=False,
                           download=True,
                           transform=transforms.ToTensor())

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=False)

train_data, test_data

100%|██████████| 9912422/9912422 [00:00<00:00, 80371998.05it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 32642331.94it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 19840070.33it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5268398.44it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



(Dataset MNIST
     Number of datapoints: 60000
     Root location: data
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: data
     Split: Test
     StandardTransform
 Transform: ToTensor())

In [8]:
# Initialize Network
model = CNN().to(device)
model

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [9]:
# Loss and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(),
                       lr=LEARNING_RATE)

loss_fn, optimizer

(CrossEntropyLoss(),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ))

In [10]:
# Train Network
model.train()
for epoch in range(EPOCHS):
  for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)
    preds = model(X)
    loss = loss_fn(preds, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [11]:
# Check accuracy on training and test data to see how good our model is
def check_accuracy(model, dataloader):

  if dataloader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on testing data")

  num_correct, num_samples = 0, 0

  model.eval()
  with torch.inference_mode():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      scores = model(X)
      _, predictions = scores.max(1)
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)

  print(f"{num_correct}/{num_samples} correct and the accuracy is {num_correct/num_samples * 100:.2f}")

check_accuracy(model, train_dataloader)
check_accuracy(model, test_dataloader)

Checking accuracy on training data
59499/60000 correct and the accuracy is 99.17
Checking accuracy on testing data
9872/10000 correct and the accuracy is 98.72
